In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as data
url="https://www.kaggle.com/datasets/fpeccia/weed-detection-in-soybean-crops"
data.download(url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: arijitmodak
Your Kaggle Key: ··········


100%|██████████| 2.37G/2.37G [01:25<00:00, 29.5MB/s]


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
IMAGE_SIZE=100
BATCH_SIZE=200
CHANNELS=3

In [ ]:
!pip install Pillow

In [ ]:
# # # Warning dont run this code This is used only for converting and tif to jpg and storing it to another folder

# from PIL import Image
# import os

# # Input and output folder paths
# input_folder = '/content/weed-detection-in-soybean-crops/dataset/broadleaf'
# output_folder = '/content/SoybeanWeedDataset/broadleaf'

# # Create the output folder if it doesn't exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # List all the files in the input folder
# tif_files = os.listdir(input_folder)

# for file in tif_files:
#     # Check if the file is a TIFF image
#     if file.lower().endswith('.tif') or file.lower().endswith('.tiff'):
#         # Load the TIFF image
#         tif_image = Image.open(os.path.join(input_folder, file))

#         # Convert the image to JPG format and save it in the output folder
#         jpg_filename = file.split('.')[0] + '.jpg'
#         tif_image.save(os.path.join(output_folder, jpg_filename), format='JPEG')

In [ ]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "/content/SoybeanWeedDataset/dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
    )
print(dataset.class_names)

Found 15336 files belonging to 4 classes.
['broadleaf', 'grass', 'soil', 'soybean']


In [ ]:
x=[]
y=[]
print(len(dataset))
for image_batch,label_batch in dataset.take(len(dataset)):
  # print(image_batch.shape)
  # print(label_batch.shape)
  for i in range(len(image_batch)):
     img=np.uint8(image_batch[i].numpy())
     x.append(img)
     y.append(label_batch[i])

77


In [ ]:
x=np.array(x)
y=np.array(y)
print(x.shape)
print(y.shape)

(15336, 100, 100, 3)
(15336,)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets,layers,models
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
classes=['broadleaf','grass','soil','soyabean']
print(len(classes))

4


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)
print(x_train.shape)
print(y_train.shape)

(12268, 100, 100, 3)
(12268,)


In [ ]:
# normalize the images
x_train=x_train/255.0
x_test=x_test/255.0

#implementing ann

cnn= models.Sequential([
    #cnn
    layers.Conv2D(filters=32,kernel_size = ( 3, 3),activation="relu",input_shape=(100,100,3)),
    layers.MaxPooling2D((2,2)),


    # dense
    layers.Flatten(),
    layers.Dense(64,activation="relu"),
    layers.Dense(4,activation="softmax")
])

cnn.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])
cnn.fit(x_train,y_train,epochs=2)

Epoch 1/2
384/384 [==============================] - 19s 13ms/step - loss: 0.4863 - accuracy: 0.8335
Epoch 2/2
384/384 [==============================] - 5s 13ms/step - loss: 0.2660 - accuracy: 0.9041


In [ ]:
cnn.evaluate(x_test,y_test)

96/96 [==============================] - 1s 7ms/step - loss: 0.2878 - accuracy: 0.8996


In [ ]:
y_pred=cnn.predict(x_test)
y_pred_classes=[np.argmax(element) for element in y_pred]

print(y_test[:5],"\t",y_pred_classes[:5])
print(accuracy_score(y_test,y_pred_classes))

96/96 [==============================] - 0s 3ms/step
[3 3 3 2 3] 	 [3, 3, 3, 2, 3]
0.8996088657105606


In [ ]:
test_img=x_test[17]
print(test_img.shape)
nx,ny,nrgb=test_img.shape
test_image=test_img.reshape(1,nx,ny,nrgb)
print(test_image.shape)
#print(test_image)
pred=cnn.predict(test_image)
pred_classes=[np.argmax(element) for element in pred]

plt.imshow(test_img)
plt.xlabel("Prediction:  "+classes[pred_classes[0]])
plt.ylabel("Actual:  "+classes[y_test[17]])

In [ ]:
image=cv2.imread("/content/trump.jpg")
#image=cv2.imread("/content/SoybeanWeed/broadleaf/1149.jpg")
image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image=cv2.resize(image,(100,100))
image=image/255.0
test_img=image.reshape(1,100,100,3)

print(image.shape)
print(test_img.shape)

(100, 100, 3)
(1, 100, 100, 3)


In [ ]:
threshold = 0.8

pred = cnn.predict(test_img)
pred_classes=[np.argmax(element) for element in pred]
npred=np.array(pred)
predp=npred.flatten()
print(predp)

max_prob = np.max(pred)
print(max_prob)
if max_prob > threshold:
    plt.imshow(image)
    plt.xlabel("Prediction:  "+classes[pred_classes[0]])
else:
    plt.imshow(image)
    plt.xlabel("Unidentified Class")